# Data Exploration (*recommendation*)

In [2]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
# import altair as alt
import numpy as np
from scipy.sparse import coo_matrix
import dask.dataframe as dd
from scipy.sparse import vstack

In [3]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [3]:
# Unzipping the file
# with zipfile.ZipFile("resources/archive.zip", 'r') as zip_ref:
#    zip_ref.extractall("resources")

In [4]:
def loading_recommendations(path):
    #df = pd.read_csv(path)
    df = dd.read_csv(path)
    df["is_recommended"] = df["is_recommended"].apply(lambda x: 1 if x == True else 0)
    return df

In [5]:
recommendations = loading_recommendations("resources/recommendations.csv")
recommendations.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,1,36.3,51580,0
1,304390,4,0,2017-02-17,0,11.5,2586,1
2,1085660,2,0,2019-11-17,1,336.5,253880,2
3,703080,0,0,2022-09-23,1,27.4,259432,3
4,526870,0,0,2021-01-10,1,7.9,23869,4


In [27]:
recommendations.shape

(41154794, 8)

In [7]:
print("There are", '{0:,.0f}'.format(recommendations["user_id"].nunique()) , "users")
print("There are", '{0:,.0f}'.format(recommendations["app_id"].nunique()) , "games")

There are 13,781,059 users
There are 37,610 games


**Average number of reviews per user**

In [8]:
review_user = recommendations[["user_id","review_id"]].groupby("user_id").count()
review_user = review_user.reset_index()
review_user.columns = [["user_id", "#_reviews"]]
review_user.head()

,user_id,#_reviews
0,0,28
1,1,1
2,2,4
3,3,2
4,4,1


In [9]:
review_user["#_reviews"].describe()

,#_reviews
count,13781059.0
mean,3.0
std,8.1
min,1.0
25%,1.0
50%,1.0
75%,3.0
max,6045.0


In [10]:
# For instance, let's take the reviews of user_id == 0
# recommendations[recommendations["user_id"]==0]

**Average number of reviews per game**

In [11]:
review_game = recommendations[["app_id","review_id"]].groupby("app_id").count()
review_game = review_game.reset_index()
review_game.columns = [["app_id", "#_reviews"]]
review_game.head()

,app_id,#_reviews
0,10,41043
1,20,4284
2,30,4432
3,40,1610
4,50,9721


In [12]:
review_game["#_reviews"].describe()

,#_reviews
count,37610.0
mean,1094.3
std,7689.3
min,1.0
25%,13.0
50%,39.0
75%,179.8
max,319492.0


In [13]:
# For instance, let's take the reviews of app_id == 10
recommendations[recommendations["app_id"]==10]

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
6175295,10,0,0,2021-12-06,1,22.6,2524579,6175295
6175299,10,0,0,2021-11-25,1,3.5,7761084,6175299
6175300,10,0,0,2021-11-25,1,5.4,9377644,6175300
6175304,10,0,0,2021-10-16,1,211.4,2713079,6175304
6175307,10,0,0,2021-07-26,1,359.6,10214996,6175307
...,...,...,...,...,...,...,...,...
38989335,10,2,0,2020-01-11,1,698.0,8010358,38989335
38989815,10,0,0,2021-07-17,1,13.0,6258273,38989815
38990023,10,2,0,2021-11-15,1,194.0,166934,38990023
38991478,10,0,0,2020-12-18,1,30.0,13780121,38991478


**User-game matrix**

In [14]:
def get_user_game_sparse_matrix(df, chunk_size, start_index, end_index, session,  sample=False, sample_size=1000):
    """
    Args::
        df::dataframe, like recommendations, approximately 41 milion of records
        chunk_size::integer, the chunk to build the pivot table
        start_index:: integer, the start of index for slicing the df
        end_index:: integer, the end of index for slicing the df
        session:: string, print start_index and end_index in the csv files
        sample:: boolean, if true use a sample of data for testing
        sample_size:: integer, the size of the sample when sample is True
    Returns::
        sparse user-matrix to save space from a pivot table with users as index and games as columns
        
    First, this function reduces the size of the data using the indexes. 
    Second, it reads data in chunks to efficiently build a pivot table with users as index and games as columns.
    Third, a pivot table is saved into a csv file
    """
    if sample:
        df = df.iloc[:sample_size]
        
    df = df.iloc[start_index:end_index+1]  # +1 to consider also the second extreme of the range

    # df = df.sort_values(by="user_id")
    # Get unique IDs to split the data into chunks
    unique_ids = df["user_id"].unique()
    
    print("The sliced dataframe has a length of", len(df))
    
    # sparse_matrix = list()
    
    # Iterate over chunks of unique IDs
    for i, ids_chunk in enumerate([unique_ids[i:i+chunk_size] for i in range(0, len(unique_ids), chunk_size)]):
        print(i+1, "  # chunk of", len(df) / chunk_size)
        # Filter the DataFrame based on the current chunk of IDs
        chunk_df = df[df['user_id'].isin(ids_chunk)]

        # Pivot the chunked DataFrame
        pivoted_chunk = pd.pivot_table(chunk_df, values='is_recommended', index='user_id', columns='app_id')
        
        pivoted_chunk.to_csv(f'assets/pivoted_chunk_{str(session)}_{i}_{ids_chunk[0]}_{ids_chunk[-1]}.csv')
        # user_game_matrix = coo_matrix(pivoted_chunk)
        # sparse_matrix.append(user_game_matrix)
    return sparse_matrix
    

In [15]:
def output_single_csv(df, df_subset_size, chunk_size):
    """
    Args:
        df::dataframe, like recommendations, approximately 41 milion of records
        df_subset_size:: integer, the size to slice the data
        chunk_size:: the size of the chunk to build the pivot table
        
    This function creates several user-games matrix calling another function to create a single
    csv file
    """
    for end in range(0, len(df)+df_subset_size, df_subset_size):
        print(end, end+df_subset_size)
        get_user_game_sparse_matrix(df, chunk_size, end, end+df_subset_size, str(end)+'-'+str(end+df_subset_size))
    return None

In [16]:
# output_single_csv(recommendations, 1000000, 10000)

In [6]:
all_cols_df = recommendations.app_id.unique().persist().compute()
all_cols_df1 = all_cols_df.to_frame().T
all_cols_df1.columns = list(all_cols_df1.loc["app_id"].values)
all_cols_df1.iloc[0] = np.NaN
all_cols_df1 = all_cols_df1.reset_index().drop('index', axis=1)
all_cols_df1.head()

,975370,304390,1085660,703080,526870,306130,238960,730,255710,289070,...,1550870,2217730,2121260,858260,1701560,1272160,1769170,1246250,510540,1643850
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
x = dd.read_csv(os.path.join("assets","pivoted_chunk_0-1000000_0_51580_9564603.csv")).set_index("user_id")
x.head(1)

,440,550,570,730,4000,22380,39210,42700,47890,105600,...,1850570,1904540,1919590,1938090,1942280,1943950,1987080,1997040,2064650,2208920
user_id,,,,,,,,,,,,,,,,,,,,,
2586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
y = dd.read_csv(os.path.join("assets","pivoted_chunk_0-1000000_10_1397203_11809734.csv")).set_index("user_id")
y.head(1)

,440,550,570,730,4000,22380,39210,42700,47890,105600,...,1846380,1850570,1904540,1919590,1938090,1943950,1987080,1997040,2064650,2208920
user_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
dd.concat([all_cols_df1, x], axis=0, ignore_index=False).persist().compute()

,975370,304390,1085660,703080,526870,306130,238960,730,255710,289070,...,1850570,1904540,1919590,1938090,1942280,1943950,1987080,1997040,2064650,2208920
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14288244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14290490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14292795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14297055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
def concat_user_game_matrices(directory):
    
     
    for i, file in enumerate(os.listdir(directory)[:5]):
        print(i,"th file of", len(os.listdir(directory)), "files")
        print(file)
        matrix = dd.read_csv(os.path.join(directory,file)).set_index("user_id")#.fillna(-1).astype("int16")
        user_game_matrix = dd.concat([all_cols_rec,matrix], axis=0, ignore_index=True).persist().compute()
    user_game_matrix = user_game_matrix.fillna(-1).astype("int16")
    return user_game_matrix

In [55]:
x1 = dd.concat([all_cols_df1, x], axis=0, ignore_index=True).persist()

In [43]:
users_games_matrix = concat_user_game_matrices("assets")

0 th file of 3377 files
pivoted_chunk_33000000-34000000_34_11062675_3557114.csv


TypeError: '<' not supported between instances of 'str' and 'int'

In [12]:
users_games_matrix

,70,400,420,440,500,620,730,4000,4500,6060,8500,8870,8930,8980,12210,17480,20920,22330,22370,22380,24010,33230,34330,39210,42700,47810,47890,49520,55150,56400,107410,202970,203160,204360,207140,209000,212500,213670,218230,218620,219150,219740,220200,220240,221100,225540,230410,231430,232090,233860,234140,236430,238320,239030,240720,242050,242760,244450,244850,251060,251570,252490,253230,255710,262060,264710,266840,268500,270880,271590,275850,281990,284160,286690,288470,289070,291550,292030,294100,298110,302670,304390,305620,307780,307960,311210,311690,311730,312660,313120,322170,322330,323190,325610,335300,349040,359320,359550,359870,361420,362890,367500,368500,371660,371970,374320,376210,377160,379430,379720,380600,381210,386180,386940,389730,391540,393380,394360,397540,403640,412020,414340,414700,418370,424030,424840,427520,433340,438100,444200,447040,453480,454650,457140,459820,466560,470220,471710,475150,476600,477160,480490,488790,489830,492720,493340,502500,504230,505460,509980,518790,526870,527230,529340,534380,546560,548430,570940,571740,573090,578080,581320,582010,582660,588030,588430,588650,590380,594330,594650,601050,601150,602960,611790,617290,620980,627270,627690,629760,632470,633230,635260,637650,640820,641320,646910,648800,653530,655500,678950,678960,686810,690790,699130,703080,719040,740130,750920,752590,753640,761890,774171,774181,774361,775500,779340,784150,787480,794260,805550,813780,814380,826630,834530,845890,848450,859580,860510,865360,870780,872670,880940,892970,911400,916440,919640,920210,924970,939960,952060,960170,975370,976310,976730,985810,985890,996580,1003590,1007040,1029690,1030840,1056640,1057090,1066780,1072420,1084600,1085660,1089980,1097840,1099410,1106840,1113560,1118200,1139900,1139980,1142710,1144200,1147560,1149620,1151340,1158310,1167630,1172470,1174180,1182900,1190000,1205520,1210320,1211020,1222680,1222690,1222700,1222730,1237970,1238000,1238040,1238810,1238860,1240440,1244460,1252330,1259420,1262350,1271700,1274570,1283400,1286580,1286680,1288320,1293830,1295510,1295920,1296610,1307550,1307890,1325200,1328670,1332010,1343370,1361510,1369630,1378990,1384160,1401590,1407200,1419170,1426210,1446780,1451940,1455840,1462040,1475810,1498570,1509960,1517290,1522820,1527950,1533390,1533420,1577120,1593500,1594320,1599340,1604030,1605220,1611600,1611910,1621690,1637320,1672970,1687950,1703340,1761390,1794680,1811260,1816300,1817190,1824220,1850570,1869590,1874880,1919590,1922560,1942280,1966900,1987080,1997040,1999360,2058190,550,9900,16900,107100,200210,205100,210970,212160,219990,221040,227300,261550,261570,268910,286160,310950,339340,364360,367520,431240,444090,460930,552100,552520,585420,601510,607080,667970,736260,757310,767560,780290,782250,782330,868270,909320,918570,933110,945360,951440,1062520,1096900,1105670,1121560,1135690,1151640,1161580,1217060,1222140,1229490,1237950,1255560,1256670,1273400,1325860,1454400,1476970,1599600,1657630,1816670,10,240,380,570,2280,2630,3590,4570,4700,6020,10180,10500,12200,15100,17470,19680,21690,22300,22320,24200,24780,24810,32440,32470,32500,34270,38420,40800,42680,47410,47780,48190,48240,48700,55230,57300,65800,65980,99900,105600,108600,109600,110800,113200,115320,200510,200710,201810,201870,203140,203770,204450,204880,207610,208200,208580,208650,209160,211820,213610,214490,214510,217200,220700,221380,221680,222480,223470,223750,228280,230190,233450,236110,236390,238460,238960,239140,241930,242550,244160,246900,248820,249130,250900,251150,252110,252530,252610,254700,257350,257510,257850,261640,265930,269210,270150,274170,274190,282070,282140,282800,285330,285900,287390,287700,289650,290340,291650,292120,292730,293760,298630,304050,304430,304650,304930,306020,311560,312520,314160,319510,319630,322500,326460,330020,332950,334230,335240,337000,339600,341800,341940,342560,343780,346010,346110,346900,352720,354140,355790,356190,363890,365360,365590,366090,367450,373420,377840,378760,382560,383120,383150,383870,383980,384180,386360,388090,391220,392160,394